## PREPARING TRAINING DATASET

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import os, fnmatch

In [2]:
def file_keeper(psg_file_path, hyp_file_path):

    data = mne.io.read_raw_edf(psg_file_path, stim_channel="Event marker", infer_types=True, preload=True)
    annotations = mne.read_annotations(hyp_file_path)

    data.filter(0.5, 30, picks=[0, 1, 2])
    data.filter(None, 5, picks=[4])
    

    return data, annotations

In [3]:
def crop_set_annotations(data, annotations):

    annotations.crop(annotations[1]["onset"] - 30*60, annotations[-2]["onset"] + 30*60)
    data.set_annotations(annotations, emit_warning=False)

    annotations_stage_id = {
        "Sleep stage W": 1,
        "Sleep stage 1": 2,
        "Sleep stage 2": 3,
        "Sleep stage 3": 4,
        "Sleep stage 4": 5,
        "Sleep stage R": 6,
    }

    events_from_the_file, event_id_info = mne.events_from_annotations(
        data, event_id= annotations_stage_id, chunk_duration= 30.0
    )

    return events_from_the_file, event_id_info, annotations_stage_id


In [4]:
def create_signals(data, events_from_the_file, annotations_id, psg_file, epochs_array):

    tmax = 30.0 - 1.0 / data.info["sfreq"]
    print(f"Number of epochs in events_from_the_file for the file: {len(events_from_the_file)}")

    epochs_from_the_file = mne.Epochs(
        raw=data,
        events=events_from_the_file,
        event_id=annotations_id,
        tmin=0.0,
        tmax=tmax,
        baseline=None,
        verbose=False        
    )
    epochs_array.append(epochs_from_the_file)

    print(f"\n  SIGNALS HAVE BEEN PREPARED FOR {psg_file.split("-")[0][0:-2]}.\n")


In [5]:
def dataset_create(files_directory):

    epochs_array = []
  
    all_files = os.listdir(files_directory)
    not_processed_files = []
    all_psg_files = []
    all_hyp_files = []

    for file in all_files:

        if file[0] == '.': # for ignoring hidden files in the directory
            continue
    
        parts = file.split("-")
    
        if parts[1] == "PSG.edf":
            all_psg_files.append(file)
        elif parts[1] == "Hypnogram.edf":
            all_hyp_files.append(file)

    # for controlling status progress
    old_process_percentage = 0
    process_percentage = 0
    iteration = 0

    for psg_file in all_psg_files:

        hyp_file = psg_file.split("-")[0][:-2] + "*" + "-Hypnogram.edf"
        possible_hyp = fnmatch.filter(all_hyp_files, hyp_file)

        if possible_hyp:

            hyp_file = possible_hyp[0]

            print(f"\n================ Files currently being processed: {psg_file}, {hyp_file} ================")

            psg_file_path = files_directory + "/" + psg_file
            hyp_file_path = files_directory + "/" + hyp_file

            data, annotations = file_keeper(psg_file_path, hyp_file_path)
            print("======== Got data and annotations. ========")

            events_from_the_file, annotations_id, _ = crop_set_annotations(data, annotations)
            print("======== Annotations cropped and set. ========")

            create_signals(data, events_from_the_file, annotations_id, psg_file, epochs_array)
            
        else:
            not_processed_file = psg_file.split("-")[0][:-2]    # Get number of the candidate, i.e. SC4812
            print(f"No such hypnogram file for {not_processed_file}")
            not_processed_files.append(not_processed_file)

        iteration += 1

        process_percentage = round(iteration / len(all_psg_files) * 100)   # process status controlling

        if process_percentage != old_process_percentage:
            print(f"======== Extracting signals data from PSG files: {process_percentage}% ========\n")
        old_process_percentage = process_percentage

    print("END. Arrays for the dataset have been prepared.")
    if not_processed_files:
        print(f"Files that weren't processed: {not_processed_files}")

    return epochs_array


## PREPARING PREDICING DATASET

In [6]:
def create_signals_predict(data, events_from_the_file, annotations_id, psg_file):

    epochs_array = []
    
    tmax = 30.0 - 1.0 / data.info["sfreq"]
    print(f"Number of epochs in events_from_the_file for the file: {len(events_from_the_file)}")

    epochs_from_the_file = mne.Epochs(
        raw=data,
        events=events_from_the_file,
        event_id=annotations_id,
        tmin=0.0,
        tmax=tmax,
        baseline=None,
        verbose=False        
    )
    epochs_array.append(epochs_from_the_file)

    print(f"\n  SIGNALS HAVE BEEN PREPARED FOR {psg_file}.\n")
    
    return epochs_array


In [7]:
def create_directories(files_names):

    directories = ()
    dir_name = "info_output"

    if not os.path.isdir(dir_name):

        main_dir = dir_name
        test_files_dir = []

        for i in files_names:
            test_files_dir.append(main_dir + "/" + i)

        directories = (main_dir, test_files_dir)

        for directory in directories:
            if isinstance(directory, list):
                for sub_directory in directory:
                    if os.path.isdir(sub_directory):
                        print(f"Folder '{sub_directory}' already exists.")
                    else:
                        os.mkdir(sub_directory)
            else:
                if os.path.isdir(directory):
                    print(f"Folder '{directory}' already exists.")
                else:
                    os.mkdir(directory)

        print("\n======== Directories has been created. ========\n")

        return main_dir
    else:
        print("Prediction output directory already exists.")
    

In [8]:
def prediction_make(x_list, y_list, model):   # x_list, y_list i.e. (5282, 43), (5282)

    print("Prediction process has been started")
    print(f"SHape of the features array: {x_list.shape}")
    print(f"SHape of the labels array: {y_list.shape}")

    correct = 0
    all_elements = x_list.shape[0]
    prediction_array = []
    true_array = []

    progress = 0

    predictions_for_epochs = model.predict(x_list)

    for element in range(all_elements):

        prediction_for_epoch = np.argmax(predictions_for_epochs[element]) + 1   # Class id, for example 1 for "Sleep stage W"
        prediction_array.append(prediction_for_epoch)

        true_result = y_list[element] + 1     # Class id, for example 1 for "Sleep stage W"
        true_array.append(true_result)

        if prediction_for_epoch == true_result:
            correct += 1

        progress = round(element/all_elements * 100, 2)
        print(f"Progress of the prediction process: {progress}%", end="\r")

    percentage = round(correct/all_elements*100, 2)
    print(f"\nFinal result is: {correct} / {all_elements} ({percentage}%)")

    return correct, all_elements, prediction_array, true_array
        

In [9]:
def create_hypnograms(features_data, labels_data, epochs_array, sfreq, img_folder_path, model, annotations_stage_id):

    # Coordinates for true hypnogram and predicted respectively
    x = []
    y = []

    x_pred = []
    y_pred = []

    percent = 0    # Variable for an exit text statistical information
    true_list = 6*[0]   # List for true numbers of the each stage, based on the annotation_stage_id
    correct_predict_list = 6*[0]   # List of the correct predictions for the each stage, based on the annotation_stage_id

    stages_names_list = list(annotations_stage_id.keys())
    stages_id_list = list(annotations_stage_id.values())

    _, _, predictions, true_array = prediction_make(features_data, labels_data, model)

    true_events = epochs_array[0].events

    for iteration in range(1, len(true_events)):

        start = int(true_events[iteration - 1][0] / sfreq)
        duration = int(true_events[iteration][0] / sfreq) - start
        stage_id = true_events[iteration - 1][2]

        current_stage_name = stages_names_list[stages_id_list.index(stage_id)]

        x.append(start)
        y.append(current_stage_name)
        x.append(start + duration)
        y.append(current_stage_name)

        true_list[stages_id_list.index(stage_id)] += 1  # counting stage elements


        # Predictions results

        predict_res = predictions[iteration - 1]   # Predicted value is a class id in the range [1, 6]

        if predict_res == stage_id:
            correct_predict_list[stages_id_list.index(predict_res)] += 1

        # print(f"Predicted value of stage id and true value - {predict_res}/{stage_id}\n\n")

        predicted_stage_name = stages_names_list[stages_id_list.index(predict_res)]

        x_pred.append(start)
        y_pred.append(predicted_stage_name)
        x_pred.append(start + duration)
        y_pred.append(predicted_stage_name)

    print("======== Coordinates for true and predicted hypnograms have been received. ========")

    # Hypnogram plotting

    stages_names_for_plot = {
        "Sleep stage 4": 0,
        "Sleep stage 3": 1,
        "Sleep stage 2": 2,
        "Sleep stage 1": 3,
        "Sleep stage R": 4,        
        "Sleep stage W": 5
    }
    positions_of_labels = [0, 1, 2, 3, 4, 5]
    y_true_data_plot = [stages_names_for_plot[elem] for elem in y]
    y_pred_data_plot = [stages_names_for_plot[elem] for elem in y_pred]

    plt.rcParams['font.family'] = 'Times New Roman'
    plt.rcParams['font.size'] = 15

    fig = plt.figure()
    fig.set_figwidth(18)
    fig.set_figheight(10)
    
    plt.subplot(2, 1, 1)
    plt.plot(x, y_true_data_plot, '-b', linewidth='1', label="True hypnogram")
    plt.yticks(positions_of_labels, stages_names_for_plot)
    plt.title(f"Hypnogram for {img_folder_path[-7:]}")
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.plot(x_pred, y_pred_data_plot, '-g', linewidth='1', label="Predicted hypnogram")
    plt.yticks(positions_of_labels, stages_names_for_plot)
    plt.legend()

    hypnogram_folder = img_folder_path + "/" + "hypnogram_info_" + img_folder_path[-7:]
    if not os.path.isdir(hypnogram_folder):
        os.mkdir(hypnogram_folder)

    path_save_img = hypnogram_folder + f"/hypnogram_{img_folder_path[-7:]}"
    plt.savefig(path_save_img)

    plt.close(fig)


    # Creating confusion map
    path_save_img = hypnogram_folder + f"/confusion_map_{img_folder_path[-7:]}"
    map = create_confusion_matrix(true_array, predictions, path_save_img)

    # Percentage of predictions
    try:
        percent = round(sum(correct_predict_list)/sum(true_list) * 100, 2)
    except ZeroDivisionError:
        percent = 0

    info = f"All prediction results for {img_folder_path[-7:]}: {sum(correct_predict_list)}/{sum(true_list)} \
                ({percent}%)\n"
    path_for_txt = hypnogram_folder + f"/predict_info_{img_folder_path[-7:]}.txt"
    with open(path_for_txt, "w") as file:
        file.write(info)

    print(info)

    for iteration in range(len(correct_predict_list)):
        try:
            percent = round(correct_predict_list[iteration]/true_list[iteration] * 100, 2)
        except ZeroDivisionError:
            percent = 0
        info = f"Prediction results for {stages_names_list[iteration]}: {correct_predict_list[iteration]}/{true_list[iteration]} \
                    ({percent}%)\n"
        print(info)

        with open(path_for_txt, "a") as file:
            file.write(info)

    print(f"======== Hypnogram {img_folder_path[-7:]} has been created. ========\n\n")     


In [10]:
def make_predict_create_hypns(model, dir_edf_files_predict):

    print(f"Summary for the NN model:\n")
    model.summary()

    # Creating directory for predictions

    files_for_predict = []
    for file in os.listdir(dir_edf_files_predict):
        name = file[0:7]
        if name not in files_for_predict:
            files_for_predict.append(name)

    dir_predict_out = create_directories(files_for_predict)

    if not dir_predict_out:
        print("Please, remove old output directory!")
        return

    # Creating hypnograms and confusion matrixes

    exit_strings = []
    
    for folder in os.listdir(dir_predict_out):

        curr_path = dir_predict_out + "/" + folder

        psg_file = folder + "*"   # name of the true psg signal files (psg and hyp) in the dir_edf_files_predict
        possible_psg_files = fnmatch.filter(os.listdir(dir_edf_files_predict), psg_file)

        psg_file_path = ""
        hyp_file_path = ""
        for file in possible_psg_files:
            if file.split('-')[1][0] == "H":
                hyp_file_path = dir_edf_files_predict + "/" + file
            else:
                psg_file_path = dir_edf_files_predict + "/" + file

        data, annotations = file_keeper(psg_file_path, hyp_file_path)
        sfreq = data.info["sfreq"]
        events_from_the_file, annotations_id, annotations_stage_id = crop_set_annotations(data, annotations)

        print(f"Current path: {curr_path}")

        epochs_array = create_signals_predict(data, events_from_the_file, annotations_id, folder)
        features_data, labels_data = feature_extract(epochs_array)
        create_hypnograms(features_data, labels_data, epochs_array, sfreq, curr_path, model, annotations_stage_id)

    print("End. All hypnograms have been created.")


## FEATURE EXTRACTION PROCESS

In [11]:
from scipy import stats
import numpy.fft as fft

# features functions for a feature extraction

def k_complex_ratio(signal):

    signal = signal * 1000000    # Convert to the microvolts

    sampling_freq = 100
    min_complex_amplitude = 75
    complex_duration = 0.5
    k_complex_count = 0

    duration_samples = int(complex_duration * sampling_freq)

    for i in range(len(signal) - duration_samples):
        segment = signal[i:i + duration_samples]
        if np.max(segment) - np.min(segment) > min_complex_amplitude:
            k_complex_count += 1

    total_samples = len(signal)
    ratio = k_complex_count / total_samples

    return ratio

def feature_mean(signal):
    mean = np.mean(signal)
    return mean

def feature_variance(signal):
    var = np.var(signal)
    return var

def feature_std_deviation(signal):
    deviation = np.std(signal)
    return deviation

def feature_max(signal):
    maximum = np.max(signal)
    return maximum

def feature_min(signal):
    minimum = np.min(signal)
    return minimum

def feature_pkp(signal):
    pkp = np.max(signal) - np.min(signal)
    return pkp

def feature_discrete_diff(signal):  # substraction from the following value the previous one
    diff = np.sum(np.abs(np.diff(signal)))
    return diff

def feature_skewness(signal):  # how values are laying relatively to the normal symmetrical distribution
    skew = stats.skew(signal, axis=-1)
    return skew
    
def feature_kurtosis(signal):  # how values are laying relatively to the mean data
    kurtosis = stats.kurtosis(signal, axis=-1)
    return kurtosis



def feature_freq_info(signal):

    freq_info = []

    sampling_freq = 100
    fft_output = np.abs(np.fft.rfft(signal))
    fft_freq = np.fft.rfftfreq(len(signal), 1/sampling_freq)

    delta = np.sum(fft_output[(fft_freq >= 1) & (fft_freq < 4)])   # Frequencies are in Hz
    freq_info.append(delta)
    theta = np.sum(fft_output[(fft_freq >= 4) & (fft_freq <= 8)])
    freq_info.append(theta)
    alpha = np.sum(fft_output[(fft_freq > 8) & (fft_freq <= 13)])
    freq_info.append(alpha)
    beta = np.sum(fft_output[(fft_freq > 13) & (fft_freq <= 30)])
    freq_info.append(beta)

    return freq_info

def feature_eye_blink(signal):

    sampling_freq = 100

    signal = signal * 1000000    # Convert to the microvolts

    blink_ampl = 380  # in microvolts
    blink_duration = 0.11 * sampling_freq
    blinks = np.where(signal > blink_ampl)[0]

    blink_times = np.diff(blinks) > blink_duration
    correct_blinks = np.sum(blink_times) + 1

    blink_frequency = correct_blinks / len(signal)

    return blink_frequency

def feature_median_of_freq(signal):

    sampling_freq = 100
    fft_output = np.abs(np.fft.rfft(signal))
    fft_freq = np.fft.rfftfreq(len(signal), 1/sampling_freq)

    important_freqs_start_from = 0.5 * np.max(fft_output)

    median = np.median(fft_freq[fft_output >= important_freqs_start_from])

    return median

def feature_rms(signal):
    rms = np.sqrt( np.sum(np.square(signal))/len(signal) )
    return rms


def features_from_eeg(eeg):

    eeg_features = []

    eeg_features.append(feature_max(eeg))
    eeg_features.append(feature_min(eeg))
    eeg_features.append(feature_mean(eeg))
    eeg_features.append(feature_variance(eeg))
    eeg_features.append(feature_std_deviation(eeg))
    eeg_features.append(feature_pkp(eeg))
    eeg_features.append(feature_discrete_diff(eeg))
    eeg_features.append(feature_skewness(eeg))
    eeg_features.append(feature_kurtosis(eeg))

    for feature in feature_freq_info(eeg):
        eeg_features.append(feature)
    eeg_features.append(k_complex_ratio(eeg))

    return eeg_features

def features_from_eog(eog):

    eog_features = []

    eog_features.append(feature_max(eog))
    eog_features.append(feature_min(eog))
    eog_features.append(feature_mean(eog))
    eog_features.append(feature_variance(eog))
    eog_features.append(feature_std_deviation(eog))
    eog_features.append(feature_pkp(eog))    
    eog_features.append(feature_eye_blink(eog))    

    return eog_features

def features_from_emg(emg):

    emg_features = []

    emg_features.append(feature_max(emg))
    emg_features.append(feature_min(emg))
    emg_features.append(feature_mean(emg))
    emg_features.append(feature_median_of_freq(emg))
    emg_features.append(feature_variance(emg))
    emg_features.append(feature_std_deviation(emg))
    emg_features.append(feature_rms(emg))
    emg_features.append(feature_pkp(emg))

    return emg_features


# x[0,1,2,3] :   0 - eeg, 1 - eeg, 2 - eog, 3 - emg

def all_features(x):
    return np.concatenate( (features_from_eeg(x[0]), features_from_eeg(x[1]), 
                            features_from_eog(x[2]), features_from_emg(x[3])), axis=-1 )

    


In [12]:
def feature_extract(epochs_array_per_file):
    
    features = []
    labels = []
    iteration = 0
    file_num = 0
    for file in epochs_array_per_file:
        class_id = file.events[:, 2]
        signals_per_epoch = file.get_data(picks=[0,1,2,4])     # 0 - eeg, 1 - eeg, 2 - eog, 4 - emg
        features_subarray = []
        for all_signals in signals_per_epoch:
            features_subarray.append(all_features(all_signals))
        features.append(features_subarray)
        labels.append(class_id)
    
    features = np.concatenate(features)
    labels = np.concatenate(labels)

    for i in range(len(labels)):     # to transform classes id's to the range [0, 5] 
        labels[i] -= 1
        
    print("\n Feature extraction process has been done.")

    return features, labels   
    

## TRAINING MODEL PROCESS

In [2]:
from keras.models import Sequential
from keras.layers import Dense

from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
def train_model_mlp(x_train, y_train, x_test, y_test, x_val, y_val):

    print(f"Train dataset shape: X {x_train.shape}, Y {y_train.shape}")
    print(f"Test dataset shape: X {x_test.shape}, Y {y_test.shape}")
    print(f"Validation dataset shape: X {x_val.shape}, Y {y_val.shape}")

    model = Sequential()
    model.add(Dense(256, input_shape=(x_train.shape[1],)))
    model.add(Dense(128))
    model.add(Dense(64))
    model.add(Dense(6, activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=30, batch_size=64, validation_data=(x_val, y_val))

    loss_value, acc_value = model.evaluate(x_test, y_test)

    print(f"Loss value: {loss_value}")
    print(f"Accuracy value: {acc_value}")
    
    print("\nModel has been trained.")
    model.save('model_mlp_end.keras')

    return model

In [16]:
def create_confusion_matrix(true_array, prediction_array, path_save_img=''):
    labels_stages = [1, 2, 3, 4, 5, 6]
    result = confusion_matrix(true_array, prediction_array, labels=labels_stages)
    print(result)

    table = pd.DataFrame(result, range(len(labels_stages)), range(len(labels_stages)))
    fig = plt.figure()
    map = sn.heatmap(table, annot=True, fmt='g')

    if path_save_img:
        plt.savefig(path_save_img)
    else:
        plt.show()
    plt.close(fig)

## EXIT CODE

Training dataset preparation

In [ ]:
print("\n\n======== CREATING TRAINING SIGNALS ========\n\n")
dir = "edf_files/sleep-cassette"
epochs_array_train = dataset_create(dir)

print("\n\n======== CREATING TEST SIGNALS ========\n\n")
dir = "edf_files/x_test_y_test"
epochs_array_test = dataset_create(dir)

print("\n\n======== CREATING VALIDATION SIGNALS ========\n\n")
dir = "edf_files/x_valid_y_valid"
epochs_array_valid = dataset_create(dir)

In [ ]:
features_train, labels_train = feature_extract(epochs_array_train)

In [ ]:
features_test, labels_test = feature_extract(epochs_array_test)

In [ ]:
features_val, labels_val = feature_extract(epochs_array_valid)

Training process

In [ ]:
model = train_model_mlp(features_train, labels_train, features_test, labels_test, features_val, labels_val)

Prediction process

In [ ]:
dir_edf_files_predict = "edf_files/x_test_y_test"
make_predict_create_hypns(model, dir_edf_files_predict)